<a href="https://colab.research.google.com/github/roman24-rca/udemy-heroes/blob/master/GEPP_v2_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from dateutil import parser
from bs4 import BeautifulSoup
import requests

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
url = 'https://downloads.apache.org/spark/' 
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)
link_files = []
for link in soup.find_all('a'):
  link_files.append(link.get('href'))
spark_link = [x for x in link_files if 'spark' in x] 
ver_spark = spark_link[1][:-1]

In [ ]:
import os
link = "https://www-us.apache.org/dist/spark/"
os.system(f"wget -q {link}{ver_spark}/{ver_spark}-bin-hadoop2.7.tgz")
os.system(f"tar xf {ver_spark}-bin-hadoop2.7.tgz")

!pip install -q pyspark

     |████████████████████████████████| 204.2MB 55kB/s 
     |████████████████████████████████| 204kB 43.0MB/s 


In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{ver_spark}-bin-hadoop2.7"

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Gepp").master("local[*]").getOrCreate()
import pyspark.sql.functions as F

#Path

In [ ]:
pathSellint = '/content/drive/My Drive/GEPP/Selint/'
pathSellout2018 = '/content/drive/My Drive/GEPP/Selout/Sellout2018r'
pathSellout2019 = '/content/drive/My Drive/GEPP/Selout/Sellout2019'
pathSellout2020 = '/content/drive/My Drive/GEPP/Selout/Sellout2020'
pathGeneral = '/content/drive/My Drive/GEPP/General'
pathSucursales = '/content/drive/My Drive/GEPP/Sucursales'
pathProductos = '/content/drive/My Drive/GEPP/Productos/productos.parquet'

#Source

In [ ]:
Sellint = spark.read.format("parquet").load(pathSellint)
Sellout2018 = spark.read.format("parquet").load(pathSellout2018)
Sellout2019 = spark.read.format("parquet").load(pathSellout2019)
Sellout2020 = spark.read.format("parquet").load(pathSellout2020)
General = spark.read.format("parquet").load(pathGeneral)
Sucursales = spark.read.format("parquet").load(pathSucursales)
productos = spark.read.format("parquet").load(pathProductos)

#Import

In [ ]:
from pyspark.sql import Row
from pyspark.sql.types import *
import pandas as pd
import numpy as np
from datetime import datetime
now = datetime.now().strftime('%Y-%m-%d')

# Segmentación Sellint

#Sellint 2018

In [ ]:
Sellint18 = Sellint.filter(F.substring("PERIODO",5,2) == 18)
Sellint18K = Sellint18.withColumn("SKU", F.concat(F.trim("MARCA"),F.trim("PRESENTACION"),F.trim("TIPO_ENVASE"), F.trim("SABOR")))
Sellint18S = Sellint18K.withColumn("date", F.when((F.col("PERIODO") == "Jan-18"), F.lit("01-18"))
.when((F.col("PERIODO") == "Feb-18"), F.lit("02-18"))
.when((F.col("PERIODO") == "Mar-18"), F.lit("03-18"))
.when((F.col("PERIODO") == "Apr-18"), F.lit("04-18"))
.when((F.col("PERIODO") == "May-18"), F.lit("05-18"))
.when((F.col("PERIODO") == "Jun-18"), F.lit("06-18"))
.when((F.col("PERIODO") == "Jul-18"), F.lit("07-18"))
.when((F.col("PERIODO") == "Aug-18"), F.lit("08-18"))
.when((F.col("PERIODO") == "Sep-18"), F.lit("09-18"))
.when((F.col("PERIODO") == "Oct-18"), F.lit("10-18"))
.when((F.col("PERIODO") == "Nov-18"), F.lit("11-18"))
.when((F.col("PERIODO") == "Dec-18"), F.lit("12-18")).otherwise(None))

In [ ]:
max_date = Sellint18S.select(F.from_unixtime(F.unix_timestamp(F.max("date"), "MM-yy"))).rdd.map(lambda x:(str(x[0]))).collect()
max_date = max_date[0][:10]
print(max_date)

2018-12-01


In [ ]:
df6 = Sellint18S.withColumn("datee", F.to_date(F.from_unixtime(F.unix_timestamp("date", "MM-yy"))).cast("date")).sort("COMPRAS", ascending=False)
df6 = df6.withColumn("amount_total", F.col("INGRESO"))

#Calculamos la recencia

In [ ]:
recency_df = df6.groupBy("SKU").agg(F.max("datee").alias("LastPurshaceDate"))
recency_df = recency_df.withColumn("date", F.lit(now))
recency_df = recency_df.withColumn("recency", F.datediff(F.to_date(recency_df["date"]),F.to_date(recency_df["LastPurshaceDate"]))).sort("recency")
recency_df = recency_df.select("SKU", "Recency")

#Eliminamos duplicados y Calculamos la frecuencia

In [ ]:
frequency_df = df6.dropDuplicates(["COMPRAS", "SKU"])
frequency_df = frequency_df.groupBy("SKU").agg(F.count("*").alias("Frequency"))
frequency_df = frequency_df.select("SKU", "Frequency")

#Calculamos el monto de ingreso

In [ ]:
monetary_df = df6.dropDuplicates(["COMPRAS", "SKU"])
monetary_df = monetary_df.groupBy("SKU").agg(F.sum("amount_total").alias("Monetary"))
monetary_df = monetary_df.select("SKU", "Monetary")

#Union de dataframe

In [ ]:
rfm_table = recency_df.join(frequency_df, "SKU").sort("SKU")
rfm_table = rfm_table.join(monetary_df, "SKU").sort("SKU")

#Metodo de Quantiles en Spark

In [ ]:
quantile = rfm_table.approxQuantile(["Recency", "Frequency", "Monetary"], [0.25, 0.5, 0.75], 0)

#Porcentaje quartil recencia

In [ ]:
rfm_table2 = rfm_table.withColumn("r_quantile", F.when((F.col("Recency") >= 0) & (F.col("Recency") <= quantile[0][0]), "4")\
                                  .when((F.col("Recency") > quantile[0][0]) & (F.col("Recency") <= quantile[0][1]), "3")\
                                  .when((F.col("Recency") > quantile[0][1]) & (F.col("Recency") <= quantile[0][2]), "2").otherwise("1"))

#Porcentaje quartil frecuencia

In [ ]:
rfm_table2 = rfm_table2.withColumn("f_quantile", F.when((F.col("Frequency") >= 0) & (F.col("Frequency") <= quantile[1][0]), "1")\
 .when((F.col("Frequency") > quantile[1][0]) & (F.col("Frequency") <= quantile[1][1]), "2")\
 .when((F.col("Frequency") > quantile[1][1]) & (F.col("Frequency") <= quantile[1][2]), "3").otherwise("4"))

#Porcentaje quartil monto

In [ ]:
rfm_table3 = rfm_table2.withColumn("m_quantile", F.when((F.col("Monetary") >= 0) & (F.col("Monetary") <= quantile[2][0]), "1")\
 .when((F.col("Monetary") > quantile[2][0]) & (F.col("Monetary") <= quantile[2][1]), "2")\
 .when((F.col("Monetary") > quantile[2][1]) & (F.col("Monetary") <= quantile[2][2]), "3").otherwise("4"))

#Funcionn UDF para RFMscore

In [ ]:
def rfm_score(r,f,m):
 value = "{0}{1}{2}".format(r,f,m)
 return value
return_rfm_score_udf = F.udf(rfm_score)
outp = rfm_table3.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

#RFM Sellint2018

In [ ]:
outp.sort("RFMScore", ascending=False).toPandas()

,SKU,Recency,Frequency,Monetary,r_quantile,f_quantile,m_quantile,RFMScore
0,7 Up3 LtsPetLima Limon,711,274,2.794359e+07,4,4,4,444
1,Mzna Sol600 MlPetManzana,711,590,8.136797e+07,4,4,4,444
2,Be Light1.5 LtsPetLimon,711,386,4.477672e+07,4,4,4,444
3,Epura1.5 LtsPetAgua Natural,711,458,2.684727e+08,4,4,4,444
4,Mzna Sol2 LtsPetManzana,711,416,5.219622e+07,4,4,4,444
...,...,...,...,...,...,...,...,...
377,CosechaPur200 MlTetra PackManzana,772,1,6.200000e+00,1,1,1,111
378,Jarritos1.5 LtsPetMultisabor,925,2,3.014700e+02,1,1,1,111
379,Sangria Casera10 OzGrbSangria,864,1,7.272000e+01,1,1,1,111
380,RockStar473 MlLataPonche,833,6,2.380869e+05,1,1,1,111


#Sellint2019

In [ ]:
Sellint19 = Sellint.filter(F.substring("PERIODO",5,2) == 19)
Sellint19K = Sellint19.withColumn("SKU", F.concat(F.trim("MARCA"),F.trim("PRESENTACION"),F.trim("TIPO_ENVASE"), F.trim("SABOR")))
Sellint19S = Sellint19K.withColumn("date", F.when((F.col("PERIODO") == "Jan-19"), F.lit("01-19"))
.when((F.col("PERIODO") == "Feb-19"), F.lit("02-19"))
.when((F.col("PERIODO") == "Mar-19"), F.lit("03-19"))
.when((F.col("PERIODO") == "Apr-19"), F.lit("04-19"))
.when((F.col("PERIODO") == "May-19"), F.lit("05-19"))
.when((F.col("PERIODO") == "Jun-19"), F.lit("06-19"))
.when((F.col("PERIODO") == "Jul-19"), F.lit("07-19"))
.when((F.col("PERIODO") == "Aug-19"), F.lit("08-19"))
.when((F.col("PERIODO") == "Sep-19"), F.lit("09-19"))
.when((F.col("PERIODO") == "Oct-19"), F.lit("10-19"))
.when((F.col("PERIODO") == "Nov-19"), F.lit("11-19"))
.when((F.col("PERIODO") == "Dec-19"), F.lit("12-19")).otherwise(None))

In [ ]:
df19 = Sellint19S.withColumn("datee", F.to_date(F.from_unixtime(F.unix_timestamp("date", "MM-yy"))).cast("date")).sort("COMPRAS", ascending=False)
df19 = df19.withColumn("amount_total", F.col("INGRESO"))

In [ ]:
recency_df19 = df19.groupBy("SKU").agg(F.max("datee").alias("LastPurshaceDate"))
recency_df19 = recency_df19.withColumn("date", F.lit(now))
recency_df19 = recency_df19.withColumn("recency", F.datediff(F.to_date(recency_df19["date"]),F.to_date(recency_df19["LastPurshaceDate"]))).sort("recency")
recency_df19 = recency_df19.select("SKU", "Recency")

In [ ]:
frequency_df19 = df19.dropDuplicates(["COMPRAS", "SKU"])
frequency_df19 = frequency_df19.groupBy("SKU").agg(F.count("*").alias("Frequency"))
frequency_df19 = frequency_df19.select("SKU", "Frequency")

In [ ]:
monetary_df19 = df19.dropDuplicates(["COMPRAS", "SKU"])
monetary_df19 = monetary_df19.groupBy("SKU").agg(F.sum("amount_total").alias("Monetary"))
monetary_df19 = monetary_df19.select("SKU", "Monetary")

In [ ]:
rfm_table19 = recency_df19.join(frequency_df19, "SKU").sort("SKU")
rfm_table19 = rfm_table19.join(monetary_df19, "SKU").sort("SKU")

In [ ]:
quantile19 = rfm_table19.approxQuantile(["Recency", "Frequency", "Monetary"], [0.25, 0.5, 0.75], 0)

In [ ]:
rfm_table219 = rfm_table19.withColumn("r_quantile", F.when((F.col("Recency") >= 0) & (F.col("Recency") <= quantile19[0][0]), "4")\
                                  .when((F.col("Recency") > quantile19[0][0]) & (F.col("Recency") <= quantile19[0][1]), "3")\
                                  .when((F.col("Recency") > quantile19[0][1]) & (F.col("Recency") <= quantile19[0][2]), "2").otherwise("1"))

In [ ]:
rfm_table219 = rfm_table219.withColumn("f_quantile", F.when((F.col("Frequency") >= 0) & (F.col("Frequency") <= quantile19[1][0]), "1")\
 .when((F.col("Frequency") > quantile19[1][0]) & (F.col("Frequency") <= quantile19[1][1]), "2")\
 .when((F.col("Frequency") > quantile19[1][1]) & (F.col("Frequency") <= quantile19[1][2]), "3").otherwise("4"))

In [ ]:
rfm_table319 = rfm_table219.withColumn("m_quantile", F.when((F.col("Monetary") >= 0) & (F.col("Monetary") <= quantile19[2][0]), "1")\
 .when((F.col("Monetary") > quantile19[2][0]) & (F.col("Monetary") <= quantile19[2][1]), "2")\
 .when((F.col("Monetary") > quantile19[2][1]) & (F.col("Monetary") <= quantile19[2][2]), "3").otherwise("4"))

In [ ]:
def rfm_score(r,f,m):
 value = "{0}{1}{2}".format(r,f,m)
 return value
return_rfm_score_udf = F.udf(rfm_score)
outp19 = rfm_table319.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

#RFM Sellint2019

In [ ]:
outp19.sort("RFMScore", ascending=False).toPandas()

,SKU,Recency,Frequency,Monetary,r_quantile,f_quantile,m_quantile,RFMScore
0,7 Up3 LtsPetLima Limon,346,284,27673973.36,4,4,4,444
1,Mzna Sol600 MlPetManzana,346,593,88057772.86,4,4,4,444
2,Be Light1.5 LtsPetLimon,346,366,54245444.12,4,4,4,444
3,Gatorade0.5 LtsPetNaranja,346,247,17988194.36,4,4,4,444
4,Mzna Sol2 LtsPetManzana,346,425,57231042.61,4,4,4,444
...,...,...,...,...,...,...,...,...
356,Be Light0.5 LtsPetManzana,649,1,45.52,1,1,1,111
357,PepsiJgoLim2 LtsPetColaJugoL,621,1,324.00,1,1,1,111
358,7 Up Limonada355 MlPetLimMin,468,2,135.78,1,1,1,111
359,Jarritos600 MlPetJamaica,649,2,500.98,1,1,1,111


#Sellint 2020

In [ ]:
Sellint20 = Sellint.filter(F.substring("PERIODO",5,2) == 20)
Sellint20K = Sellint20.withColumn("SKU", F.concat(F.trim("MARCA"),F.trim("PRESENTACION"),F.trim("TIPO_ENVASE"), F.trim("SABOR")))
Sellint20S = Sellint20K.withColumn("date", F.when((F.col("PERIODO") == "Jan-20"), F.lit("01-20"))
.when((F.col("PERIODO") == "Feb-20"), F.lit("02-20"))
.when((F.col("PERIODO") == "Mar-20"), F.lit("03-20"))
.when((F.col("PERIODO") == "Apr-20"), F.lit("04-20"))
.when((F.col("PERIODO") == "May-20"), F.lit("05-20"))
.when((F.col("PERIODO") == "Jun-20"), F.lit("06-20"))
.when((F.col("PERIODO") == "Jul-20"), F.lit("07-20"))
.when((F.col("PERIODO") == "Aug-20"), F.lit("08-20"))
.when((F.col("PERIODO") == "Sep-20"), F.lit("09-20"))
.when((F.col("PERIODO") == "Oct-20"), F.lit("10-20"))
.when((F.col("PERIODO") == "Nov-20"), F.lit("11-20"))
.when((F.col("PERIODO") == "Dec-20"), F.lit("12-20")).otherwise(None))

In [ ]:
df6 = Sellint20S.withColumn("datee", F.to_date(F.from_unixtime(F.unix_timestamp("date", "MM-yy"))).cast("date")).sort("COMPRAS", ascending=False)
df6 = df6.withColumn("amount_total", F.col("INGRESO"))

In [ ]:
recency_df = df6.groupBy("SKU").agg(F.max("datee").alias("LastPurshaceDate"))
recency_df = recency_df.withColumn("date", F.lit(now))
recency_df = recency_df.withColumn("recency", F.datediff(F.to_date(recency_df["date"]),F.to_date(recency_df["LastPurshaceDate"]))).sort("recency")
recency_df = recency_df.select("SKU", "Recency")

In [ ]:
frequency_df = df6.dropDuplicates(["COMPRAS", "SKU"])
frequency_df = frequency_df.groupBy("SKU").agg(F.count("*").alias("Frequency"))
frequency_df = frequency_df.select("SKU", "Frequency")

In [ ]:
monetary_df = df6.dropDuplicates(["COMPRAS", "SKU"])
monetary_df = monetary_df.groupBy("SKU").agg(F.sum("amount_total").alias("Monetary"))
monetary_df = monetary_df.select("SKU", "Monetary")

In [ ]:
rfm_table = recency_df.join(frequency_df, "SKU").sort("SKU")
rfm_table = rfm_table.join(monetary_df, "SKU").sort("SKU")

In [ ]:
quantile = rfm_table.approxQuantile(["Recency", "Frequency", "Monetary"], [0.25, 0.5, 0.75], 0)

In [ ]:
rfm_table2 = rfm_table.withColumn("r_quantile", F.when((F.col("Recency") >= 0) & (F.col("Recency") <= quantile[0][0]), "4")\
                                  .when((F.col("Recency") > quantile[0][0]) & (F.col("Recency") <= quantile[0][1]), "3")\
                                  .when((F.col("Recency") > quantile[0][1]) & (F.col("Recency") <= quantile[0][2]), "2").otherwise("1"))

In [ ]:
rfm_table2 = rfm_table2.withColumn("f_quantile", F.when((F.col("Frequency") >= 0) & (F.col("Frequency") <= quantile[1][0]), "1")\
 .when((F.col("Frequency") > quantile[1][0]) & (F.col("Frequency") <= quantile[1][1]), "2")\
 .when((F.col("Frequency") > quantile[1][1]) & (F.col("Frequency") <= quantile[1][2]), "3").otherwise("4"))

In [ ]:
rfm_table20 = rfm_table2.withColumn("m_quantile", F.when((F.col("Monetary") >= 0) & (F.col("Monetary") <= quantile[2][0]), "1")\
 .when((F.col("Monetary") > quantile[2][0]) & (F.col("Monetary") <= quantile[2][1]), "2")\
 .when((F.col("Monetary") > quantile[2][1]) & (F.col("Monetary") <= quantile[2][2]), "3").otherwise("4"))

In [ ]:
def rfm_score(r,f,m):
 value = "{0}{1}{2}".format(r,f,m)
 return value
return_rfm_score_udf = F.udf(rfm_score)
outp = rfm_table20.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

#RFM Sellint2020

In [ ]:
outp.sort("RFMScore", ascending=False).toPandas()

,SKU,Recency,Frequency,Monetary,r_quantile,f_quantile,m_quantile,RFMScore
0,7 Up3 LtsPetLima Limon,71,244,2.030724e+07,4,4,4,444
1,Mzna Sol600 MlPetManzana,71,473,6.007878e+07,4,4,4,444
2,Be Light1.5 LtsPetLimon,71,334,4.874636e+07,4,4,4,444
3,Epura1.5 LtsPetAgua Natural,71,390,2.103038e+08,4,4,4,444
4,Mzna Sol2 LtsPetManzana,71,357,3.157846e+07,4,4,4,444
...,...,...,...,...,...,...,...,...
338,Gatorade70 GrStyroluxPonche,133,2,4.853757e+04,1,1,1,111
339,Jarritos1.5 LtsPetMultisabor,315,1,1.245100e+02,1,1,1,111
340,Sangria Casera10 OzGrbSangria,194,1,1.488000e+02,1,1,1,111
341,Manzanada600 MlPetManzanada,284,2,3.240000e+02,1,1,1,111


#Segmentación Sellout

#Sellout2018

In [ ]:
SelloutProductos18 = Sellout2018.join(productos, Sellout2018.UPC == productos.CODIGOBARRAS,  "inner")

In [ ]:
Sellout18K = SelloutProductos18.withColumn("SKU", F.concat(F.trim("MARCA"),F.trim("PRESENTACION"),F.trim("TIPO_ENVASE"), F.trim("SABOR")))
Sellout18M = Sellout18K.withColumn("mes", F.substring("FECHA",4,3))
Sellint18S = Sellout18M.withColumn("mm", F.when((F.col("mes") == "JAN"), F.lit("01"))
.when((F.col("mes") == "FEB"), F.lit("02"))
.when((F.col("mes") == "MAR"), F.lit("03"))
.when((F.col("mes") == "APR"), F.lit("04"))
.when((F.col("mes") == "MAY"), F.lit("05"))
.when((F.col("mes") == "JUN"), F.lit("06"))
.when((F.col("mes") == "JUL"), F.lit("07"))
.when((F.col("mes") == "AUG"), F.lit("08"))
.when((F.col("mes") == "SEP"), F.lit("09"))
.when((F.col("mes") == "OCT"), F.lit("10"))
.when((F.col("mes") == "NOV"), F.lit("11"))
.when((F.col("mes") == "DEC"), F.lit("12")).otherwise(None))
Sellint18F = Sellint18S.withColumn("FECHA1", F.concat(F.substring("FECHA",1,3),"mm",F.substring("FECHA",7,3)))

In [ ]:
df6 = Sellint18F.withColumn("datee", F.to_date(F.from_unixtime(F.unix_timestamp("FECHA1", "dd-MM-yy"))).cast("date")).sort("VENTA_UNIDADES", ascending=False)
df6 = df6.withColumn("amount_total", F.col("Venta_IMPORTE"))

In [ ]:
recency_df = df6.groupBy("SKU").agg(F.max("datee").alias("LastPurshaceDate"))
recency_df = recency_df.withColumn("date", F.lit(now))
recency_df = recency_df.withColumn("recency", F.datediff(F.to_date(recency_df["date"]),F.to_date(recency_df["LastPurshaceDate"]))).sort("recency")
recency_df = recency_df.select("SKU", "Recency")

In [ ]:
frequency_df = df6.dropDuplicates(["VENTA_UNIDADES", "SKU"])
frequency_df = frequency_df.groupBy("SKU").agg(F.count("*").alias("Frequency"))
frequency_df = frequency_df.select("SKU", "Frequency")

In [ ]:
monetary_df = df6.dropDuplicates(["VENTA_UNIDADES", "SKU"])
monetary_df = monetary_df.groupBy("SKU").agg(F.sum("amount_total").alias("Monetary"))
monetary_df = monetary_df.select("SKU", "Monetary")

In [ ]:
rfm_table = recency_df.join(frequency_df, "SKU").sort("SKU")
rfm_table = rfm_table.join(monetary_df, "SKU").sort("SKU")

In [ ]:
quantile = rfm_table.approxQuantile(["Recency", "Frequency", "Monetary"], [0.25, 0.5, 0.75], 0)

In [ ]:
rfm_table2 = rfm_table.withColumn("r_quantile", F.when((F.col("Recency") >= 0) & (F.col("Recency") <= quantile[0][0]), "4")\
                                  .when((F.col("Recency") > quantile[0][0]) & (F.col("Recency") <= quantile[0][1]), "3")\
                                  .when((F.col("Recency") > quantile[0][1]) & (F.col("Recency") <= quantile[0][2]), "2").otherwise("1"))

In [ ]:
rfm_table2 = rfm_table2.withColumn("f_quantile", F.when((F.col("Frequency") >= 0) & (F.col("Frequency") <= quantile[1][0]), "1")\
 .when((F.col("Frequency") > quantile[1][0]) & (F.col("Frequency") <= quantile[1][1]), "2")\
 .when((F.col("Frequency") > quantile[1][1]) & (F.col("Frequency") <= quantile[1][2]), "3").otherwise("4"))

In [ ]:
rfm_tableo18 = rfm_table2.withColumn("m_quantile", F.when((F.col("Monetary") >= 0) & (F.col("Monetary") <= quantile[2][0]), "1")\
 .when((F.col("Monetary") > quantile[2][0]) & (F.col("Monetary") <= quantile[2][1]), "2")\
 .when((F.col("Monetary") > quantile[2][1]) & (F.col("Monetary") <= quantile[2][2]), "3").otherwise("4"))

In [ ]:
def rfm_score(r,f,m):
 value = "{0}{1}{2}".format(r,f,m)
 return value
return_rfm_score_udf = F.udf(rfm_score)
outp = rfm_tableo18.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

# RFM Sellout 2018

In [ ]:
outp.sort("RFMScore", ascending=False).toPandas()

,SKU,Recency,Frequency,Monetary,r_quantile,f_quantile,m_quantile,RFMScore
0,7 Up3 LtsPetLima Limon,681,136,182890.1453,4,4,4,444
1,Mzna Sol600 MlPetManzana,681,268,410903.0367,4,4,4,444
2,Be Light1.5 LtsPetLimon,681,189,158780.6976,4,4,4,444
3,Mzna Sol2 LtsPetManzana,681,144,213524.3702,4,4,4,444
4,Gatorade600 MlPetLima Limon,681,242,507520.3428,4,4,4,444
...,...,...,...,...,...,...,...,...
357,Ok600 MlPetDurazno,689,1,4.3100,1,1,1,111
358,CosechaPur200 MlTetra PackManzana,949,1,3.1000,1,1,1,111
359,Aguas Frescas3 LtsPetJamaica,780,1,7.7400,1,1,1,111
360,JumexFresh1.5 LtsPetConga,900,1,0.6500,1,1,1,111


#Sellout2019

In [ ]:
SelloutProductos19 = Sellout2019.join(productos, Sellout2019.UPC == productos.CODIGOBARRAS,  "inner")

In [ ]:
Sellout19K = SelloutProductos19.withColumn("SKU", F.concat(F.trim("MARCA"),F.trim("PRESENTACION"),F.trim("TIPO_ENVASE"), F.trim("SABOR")))
Sellout19M = Sellout19K.withColumn("mes", F.substring("FECHA",4,3))
Sellint19S = Sellout19M.withColumn("mm", F.when((F.col("mes") == "JAN"), F.lit("01"))
.when((F.col("mes") == "FEB"), F.lit("02"))
.when((F.col("mes") == "MAR"), F.lit("03"))
.when((F.col("mes") == "APR"), F.lit("04"))
.when((F.col("mes") == "MAY"), F.lit("05"))
.when((F.col("mes") == "JUN"), F.lit("06"))
.when((F.col("mes") == "JUL"), F.lit("07"))
.when((F.col("mes") == "AUG"), F.lit("08"))
.when((F.col("mes") == "SEP"), F.lit("09"))
.when((F.col("mes") == "OCT"), F.lit("10"))
.when((F.col("mes") == "NOV"), F.lit("11"))
.when((F.col("mes") == "DEC"), F.lit("12")).otherwise(None))
Sellint19F = Sellint19S.withColumn("FECHA1", F.concat(F.substring("FECHA",1,3),"mm",F.substring("FECHA",7,3)))

In [ ]:
df6 = Sellint19F.withColumn("datee", F.to_date(F.from_unixtime(F.unix_timestamp("FECHA1", "dd-MM-yy"))).cast("date")).sort("VENTA_UNIDADES", ascending=False)
df6 = df6.withColumn("amount_total", F.col("Venta_IMPORTE"))

In [ ]:
recency_df = df6.groupBy("SKU").agg(F.max("datee").alias("LastPurshaceDate"))
recency_df = recency_df.withColumn("date", F.lit(now))
recency_df = recency_df.withColumn("recency", F.datediff(F.to_date(recency_df["date"]),F.to_date(recency_df["LastPurshaceDate"]))).sort("recency")
recency_df = recency_df.select("SKU", "Recency")

In [ ]:
frequency_df = df6.dropDuplicates(["VENTA_UNIDADES", "SKU"])
frequency_df = frequency_df.groupBy("SKU").agg(F.count("*").alias("Frequency"))
frequency_df = frequency_df.select("SKU", "Frequency")

In [ ]:
monetary_df = df6.dropDuplicates(["VENTA_UNIDADES", "SKU"])
monetary_df = monetary_df.groupBy("SKU").agg(F.sum("amount_total").alias("Monetary"))
monetary_df = monetary_df.select("SKU", "Monetary")

In [ ]:
rfm_table = recency_df.join(frequency_df, "SKU").sort("SKU")
rfm_table = rfm_table.join(monetary_df, "SKU").sort("SKU")

In [ ]:
quantile = rfm_table.approxQuantile(["Recency", "Frequency", "Monetary"], [0.25, 0.5, 0.75], 0)

In [ ]:
rfm_table2 = rfm_table.withColumn("r_quantile", F.when((F.col("Recency") >= 0) & (F.col("Recency") <= quantile[0][0]), "4")\
                                  .when((F.col("Recency") > quantile[0][0]) & (F.col("Recency") <= quantile[0][1]), "3")\
                                  .when((F.col("Recency") > quantile[0][1]) & (F.col("Recency") <= quantile[0][2]), "2").otherwise("1"))

In [ ]:
rfm_table2 = rfm_table2.withColumn("f_quantile", F.when((F.col("Frequency") >= 0) & (F.col("Frequency") <= quantile[1][0]), "1")\
 .when((F.col("Frequency") > quantile[1][0]) & (F.col("Frequency") <= quantile[1][1]), "2")\
 .when((F.col("Frequency") > quantile[1][1]) & (F.col("Frequency") <= quantile[1][2]), "3").otherwise("4"))

In [ ]:
rfm_table019 = rfm_table2.withColumn("m_quantile", F.when((F.col("Monetary") >= 0) & (F.col("Monetary") <= quantile[2][0]), "1")\
 .when((F.col("Monetary") > quantile[2][0]) & (F.col("Monetary") <= quantile[2][1]), "2")\
 .when((F.col("Monetary") > quantile[2][1]) & (F.col("Monetary") <= quantile[2][2]), "3").otherwise("4"))

In [ ]:
def rfm_score(r,f,m):
 value = "{0}{1}{2}".format(r,f,m)
 return value
return_rfm_score_udf = F.udf(rfm_score)
outp = rfm_table019.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

# RFM Sellout 2019

In [ ]:
outp.sort("RFMScore", ascending=False).toPandas()

,SKU,Recency,Frequency,Monetary,r_quantile,f_quantile,m_quantile,RFMScore
0,7 Up3 LtsPetLima Limon,316,138,198646.6943,4,4,4,444
1,Lipton Light600 MlPetTé Verde Light,316,183,145693.1850,4,4,4,444
2,Mzna Sol600 MlPetManzana,316,240,323493.0243,4,4,4,444
3,Epura1.5 LtsPetAgua Natural,316,198,212924.4235,4,4,4,444
4,Mzna Sol2 LtsPetManzana,316,155,178741.3112,4,4,4,444
...,...,...,...,...,...,...,...,...
353,PepsiJgoLim400 MlPetColaJugoL,331,11,302.3595,1,1,1,111
354,Aguas Frescas3 LtsPetJamaica,456,1,3.8700,1,1,1,111
355,Jarritos600 MlPetCitrus,387,4,60.9700,1,1,1,111
356,Titan2.5 LtsPetTuti-Fruti,364,1,40.5000,1,1,1,111


# Sellout 2020

In [ ]:
SelloutProductos20 = Sellout2020.join(productos, Sellout2020.UPC == productos.CODIGOBARRAS,  "inner")

In [ ]:
Sellout20K = SelloutProductos20.withColumn("SKU", F.concat(F.trim("MARCA"),F.trim("PRESENTACION"),F.trim("TIPO_ENVASE"), F.trim("SABOR")))
Sellout20M = Sellout20K.withColumn("mes", F.substring("FECHA",4,3))
Sellint20S = Sellout20M.withColumn("mm", F.when((F.col("mes") == "JAN"), F.lit("01"))
.when((F.col("mes") == "FEB"), F.lit("02"))
.when((F.col("mes") == "MAR"), F.lit("03"))
.when((F.col("mes") == "APR"), F.lit("04"))
.when((F.col("mes") == "MAY"), F.lit("05"))
.when((F.col("mes") == "JUN"), F.lit("06"))
.when((F.col("mes") == "JUL"), F.lit("07"))
.when((F.col("mes") == "AUG"), F.lit("08"))
.when((F.col("mes") == "SEP"), F.lit("09"))
.when((F.col("mes") == "OCT"), F.lit("10"))
.when((F.col("mes") == "NOV"), F.lit("11"))
.when((F.col("mes") == "DEC"), F.lit("12")).otherwise(None))
Sellint20F = Sellint20S.withColumn("FECHA1", F.concat(F.substring("FECHA",1,3),"mm",F.substring("FECHA",7,3)))

In [ ]:
df6 = Sellint20F.withColumn("datee", F.to_date(F.from_unixtime(F.unix_timestamp("FECHA1", "dd-MM-yy"))).cast("date")).sort("VENTA_UNIDADES", ascending=False)
df6 = df6.withColumn("amount_total", F.col("Venta_IMPORTE"))

In [ ]:
recency_df = df6.groupBy("SKU").agg(F.max("datee").alias("LastPurshaceDate"))
recency_df = recency_df.withColumn("date", F.lit(now))
recency_df = recency_df.withColumn("recency", F.datediff(F.to_date(recency_df["date"]),F.to_date(recency_df["LastPurshaceDate"]))).sort("recency")
recency_df = recency_df.select("SKU", "Recency")

In [ ]:
frequency_df = df6.dropDuplicates(["VENTA_UNIDADES", "SKU"])
frequency_df = frequency_df.groupBy("SKU").agg(F.count("*").alias("Frequency"))
frequency_df = frequency_df.select("SKU", "Frequency")

In [ ]:
monetary_df = df6.dropDuplicates(["VENTA_UNIDADES", "SKU"])
monetary_df = monetary_df.groupBy("SKU").agg(F.sum("amount_total").alias("Monetary"))
monetary_df = monetary_df.select("SKU", "Monetary")

In [ ]:
rfm_table = recency_df.join(frequency_df, "SKU").sort("SKU")
rfm_table = rfm_table.join(monetary_df, "SKU").sort("SKU")

In [ ]:
quantile = rfm_table.approxQuantile(["Recency", "Frequency", "Monetary"], [0.25, 0.5, 0.75], 0)

In [ ]:
rfm_table2 = rfm_table.withColumn("r_quantile", F.when((F.col("Recency") >= 0) & (F.col("Recency") <= quantile[0][0]), "4")\
                                  .when((F.col("Recency") > quantile[0][0]) & (F.col("Recency") <= quantile[0][1]), "3")\
                                  .when((F.col("Recency") > quantile[0][1]) & (F.col("Recency") <= quantile[0][2]), "2").otherwise("1"))

In [ ]:
rfm_table2 = rfm_table2.withColumn("f_quantile", F.when((F.col("Frequency") >= 0) & (F.col("Frequency") <= quantile[1][0]), "1")\
 .when((F.col("Frequency") > quantile[1][0]) & (F.col("Frequency") <= quantile[1][1]), "2")\
 .when((F.col("Frequency") > quantile[1][1]) & (F.col("Frequency") <= quantile[1][2]), "3").otherwise("4"))

In [ ]:
rfm_table020 = rfm_table2.withColumn("m_quantile", F.when((F.col("Monetary") >= 0) & (F.col("Monetary") <= quantile[2][0]), "1")\
 .when((F.col("Monetary") > quantile[2][0]) & (F.col("Monetary") <= quantile[2][1]), "2")\
 .when((F.col("Monetary") > quantile[2][1]) & (F.col("Monetary") <= quantile[2][2]), "3").otherwise("4"))

In [ ]:
def rfm_score(r,f,m):
 value = "{0}{1}{2}".format(r,f,m)
 return value
return_rfm_score_udf = F.udf(rfm_score)
outp = rfm_table020.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

# RFM Sellout 2020

In [ ]:
outp.sort("RFMScore", ascending=False).toPandas()

,SKU,Recency,Frequency,Monetary,r_quantile,f_quantile,m_quantile,RFMScore
0,Lipton Light600 MlPetTé Verde Light,256,118,79585.1774,4,4,4,444
1,Mzna Sol600 MlPetManzana,256,168,137283.7760,4,4,4,444
2,Epura1.5 LtsPetAgua Natural,256,116,82267.5108,4,4,4,444
3,Mzna Sol2 LtsPetManzana,256,114,158451.7543,4,4,4,444
4,Pepsi Black12 OzPetCola Black,256,252,138934.4107,4,4,4,444
...,...,...,...,...,...,...,...,...
301,Be Light1 LtsPetKiwi-Fresa,261,2,9.4800,1,1,1,111
302,JumexFresh1.5 LtsPetConga,281,2,15.0500,1,1,1,111
303,Agua Glacial1 GalonPetAgua Natural,261,16,1980.0000,1,1,1,111
304,Jarritos600 MlPetCitrus,296,1,7.8700,1,1,1,111


# Write Territorio SKU y Score

In [ ]:
outp18 = rfm_table3.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

In [ ]:
Sellint18Agg = Sellint18K.select("CADENA_COMERCIAL","SKU").distinct()
SucursalesAgg = Sucursales.select("LLAVE", "REGION", "TERRITORIO", "CADENA_COMERCIAL").distinct()
SellintIndi = Sellint18Agg.join(SucursalesAgg, "CADENA_COMERCIAL")
Sellint18Suc = SellintIndi.join(outp, "SKU").select("LLAVE", "REGION", "TERRITORIO","CADENA_COMERCIAL","SKU","RFMScore").withColumn("PERIODO", F.lit("2018")).withColumn("SUCURSAL", F.col("LLAVE"))
Sellint18Sucb = Sellint18Suc.select("PERIODO","REGION", "TERRITORIO", "CADENA_COMERCIAL", "SUCURSAL", "SKU", "RFMScore")


In [ ]:
Sellint18Sucb.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('/content/drive/My Drive/GEPP/Resultados/ouSellint18')

In [ ]:
outp19 = rfm_table319.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

In [ ]:
Sellint19Agg = Sellint19K.select("CADENA_COMERCIAL","SKU").distinct()
SucursalesAgg = Sucursales.select("LLAVE", "REGION", "TERRITORIO", "CADENA_COMERCIAL").distinct()
SellintIndi = Sellint19Agg.join(SucursalesAgg, "CADENA_COMERCIAL")
Sellint19Suc = SellintIndi.join(outp, "SKU").select("LLAVE", "REGION", "TERRITORIO","CADENA_COMERCIAL","SKU","RFMScore").withColumn("PERIODO", F.lit("2019")).withColumn("SUCURSAL", F.col("LLAVE"))
Sellint19Sucb = Sellint19Suc.select("PERIODO","REGION", "TERRITORIO", "CADENA_COMERCIAL", "SUCURSAL", "SKU", "RFMScore")

In [ ]:
Sellint19Sucb.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('/content/drive/My Drive/GEPP/Resultados/ouSellint19')

In [ ]:
outp20 = rfm_table20.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

In [ ]:
Sellint20Agg = Sellint20K.select("CADENA_COMERCIAL","SKU").distinct()
SucursalesAgg = Sucursales.select("LLAVE", "REGION", "TERRITORIO", "CADENA_COMERCIAL").distinct()
SellintIndi = Sellint20Agg.join(SucursalesAgg, "CADENA_COMERCIAL")
Sellint20Suc = SellintIndi.join(outp, "SKU").select("LLAVE", "REGION", "TERRITORIO","CADENA_COMERCIAL","SKU","RFMScore").withColumn("PERIODO", F.lit("2020")).withColumn("SUCURSAL", F.col("LLAVE"))
Sellint20Sucb = Sellint20Suc.select("PERIODO","REGION", "TERRITORIO", "CADENA_COMERCIAL", "SUCURSAL", "SKU", "RFMScore")

In [ ]:
Sellint20Sucb.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('/content/drive/My Drive/GEPP/Resultados/ouSellint20')

In [ ]:
ouSellint1819 = Sellint18Sucb.unionAll(Sellint19Sucb)
ouSellint = ouSellint1819.unionAll(Sellint20Sucb)
Sellint18Sucb.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('/content/drive/My Drive/GEPP/Resultados/ouSellint')

In [ ]:
outpp18 = rfm_tableo18.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))

In [ ]:
Sucursalesk18 = Sellout18K.withColumn("LLAVE", F.concat(F.trim("NUMERO_DE_TIENDA"), F.trim("NOMBRE_DE_TIENDA")))

In [ ]:
Sellout18Agg = Sucursalesk18.select("LLAVE","SKU").distinct()
SucursalesAgg = Sucursales.select("LLAVE", "REGION", "TERRITORIO", "CADENA_COMERCIAL").distinct()
SellintIndi = Sellout18Agg.join(SucursalesAgg, "LLAVE")
Sellout18Suc = SellintIndi.join(outpp18, "SKU").select("LLAVE", "REGION", "TERRITORIO", "CADENA_COMERCIAL","SKU","RFMScore").withColumn("PERIODO", F.lit("2018")).withColumn("SUCURSAL", F.col("LLAVE"))
Sellout18Sucb = Sellint18Suc.select("PERIODO","REGION", "TERRITORIO", "CADENA_COMERCIAL", "SUCURSAL", "SKU", "RFMScore")

In [ ]:
Sellout18Sucb.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('/content/drive/My Drive/GEPP/Resultados/ouSellout18')

In [ ]:
outpp19 = rfm_table019.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))
Sucursalesk19 = Sellout19K.withColumn("LLAVE", F.concat(F.trim("NUMERO_DE_TIENDA"), F.trim("NOMBRE_DE_TIENDA")))

In [ ]:
Sellout19Agg = Sucursalesk19.select("LLAVE","SKU").distinct()
SucursalesAgg = Sucursales.select("LLAVE", "REGION", "TERRITORIO", "CADENA_COMERCIAL").distinct()
SellintIndi = Sellout19Agg.join(SucursalesAgg, "LLAVE")
Sellout19Suc = SellintIndi.join(outpp19, "SKU").select("LLAVE", "REGION", "TERRITORIO", "CADENA_COMERCIAL","SKU","RFMScore").withColumn("PERIODO", F.lit("2019")).withColumn("SUCURSAL", F.col("LLAVE"))
Sellout19Sucb = Sellint19Suc.select("PERIODO","REGION", "TERRITORIO", "CADENA_COMERCIAL", "SUCURSAL", "SKU", "RFMScore")

In [ ]:
Sellout19Sucb.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('/content/drive/My Drive/GEPP/Resultados/ouSellout19')

In [ ]:
outpp20 = rfm_table020.withColumn("RFMScore", return_rfm_score_udf(F.col("r_quantile"),F.col("f_quantile"),F.col("m_quantile")))
Sucursalesk20 = Sellout20K.withColumn("LLAVE", F.concat(F.trim("NUMERO_DE_TIENDA"), F.trim("NOMBRE_DE_TIENDA")))

In [ ]:
Sellout20Agg = Sucursalesk20.select("LLAVE","SKU").distinct()
SucursalesAgg = Sucursales.select("LLAVE", "REGION", "TERRITORIO", "CADENA_COMERCIAL").distinct()
SellintIndi = Sellout20Agg.join(SucursalesAgg, "LLAVE")
Sellout20Suc = SellintIndi.join(outpp20, "SKU").select("LLAVE", "REGION", "TERRITORIO", "CADENA_COMERCIAL", "SKU","RFMScore").withColumn("PERIODO", F.lit("2020")).withColumn("SUCURSAL", F.col("LLAVE"))
Sellout20Sucb = Sellint20Suc.select("PERIODO","REGION", "TERRITORIO", "CADENA_COMERCIAL", "SUCURSAL", "SKU", "RFMScore")

In [ ]:
Sellout20Sucb.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('/content/drive/My Drive/GEPP/Resultados/ouSellout20')

In [ ]:
ouSellout1819 = Sellout18Sucb.unionAll(Sellout19Sucb)
ouSellout = ouSellout1819.unionAll(Sellout20Sucb)
ouSellout.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('/content/drive/My Drive/GEPP/Resultados/ouSellout')